# Chapter 12. Geospatial Data

**Geospatial data** is data that is associated with spatial locations, usually locations on Earth. For example, the percentage of people in each state who voted for a presidential candidate is an example of geospatial data, as are the locations of earthquake epicenters. In this chapter, we will discuss techniques for analyzing and visualizing geographic data.

# 12.1 Map Projections and Distance Metrics

## Longitude and Latitude

Consider the location below in the middle of the Atlantic Ocean, indicated by a red point. How would we describe this location of this point?

![](img/coordinate.png)

The Earth is divided into Northern and Southern Hemispheres by the **equator** and into Eastern and Western Hemispheres by the **prime meridian**. Locations on the surface of the earth can be specified as angles relative to the equator and prime meridian. 

![](img/coordinate_labeled.png)

The angle relative to the equator is called the **latitude**. This point is $30^\circ$ north of the equator, so the latitude is $\phi = 30^\circ$. If it had been south of the equator, then the angle would have been negative. The latitude ranges from $-90^\circ$ (the South Pole) to $90^\circ$ (the North Pole).

The angle relative to the prime meridian is called the **longitude**. This point is $45^\circ$ west of the prime meridian, so the longitude is $\lambda = -45^\circ$. If it had been east of the prime meridian, then the angle would have been positive. The longitude ranges from $-180^\circ$ to $180^\circ$.

## Distance Metrics

How do we measure the distance between two points, ${\bf x}_1 = (\lambda_1, \phi_1)$ and ${\bf x}_2 = (\lambda_2, \phi_2)$, on the surface of the Earth? The obvious choice, Euclidean distance, 
$$ d({\bf x}_1, {\bf x}_2) = \sqrt{(\lambda_1 - \lambda_2)^2 + (\phi_1 - \phi_2)^2} $$
is unsatisfactory. To see why, consider the distance between Vancouver, Canada ($-123.1838^\circ$, $49.1947^\circ$) and London, United Kingdom ($-0.4613^\circ$, $51.4775^\circ$). Euclidean distance would make these two cities are very far apart, about as far apart as San Francisco and Madrid. But as you might know if you have ever taken a transatlantic flight, the distance can be shorter if you fly north. The shortest path between Vancouver and London is shown below.

![](img/haversine.png)

The distance between two points on the surface of the Earth is the distance of the _shortest path_ between them. On a sphere, the shortest path between any two points is the path along the _great circle_---that is, the circle passing through the two points, whose diameter matches the diameter of the sphere. The great circle is represented by a dashed line in the figure above.

The distance along the great circle is known as **Haversine distance**. Haversine distance is calculated as 
$$ d({\bf x}_1, {\bf x}_2) = 2r \arcsin\left( \sqrt{\sin^2\left( \frac{\phi_1 - \phi_2}{2} \right) + \cos(\phi_1) \cos(\phi_2) \sin^2\left( \frac{\lambda_1 - \lambda_2}{2} \right)} \right),$$
where $r$ is the radius of the sphere. 

In [ ]:
import numpy as np

# Mean radius of the earth (in km)
EARTH_RADIUS = 6371.009

def haversine(point1, point2):
    """
    Calculate the great circle distance between two points
    on the Earth, specified as (lon, lat), where lon and lat
    are in degrees.
    
    Returns: distance between points in km
    """
    # convert decimal degrees to radians
    lon1, lat1 = [np.radians(x) for x in point1]
    lon2, lat2 = [np.radians(x) for x in point2]

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    return 2 * EARTH_RADIUS * np.arcsin(np.sqrt(a))

In [ ]:
vancouver = (-123.1838, 49.1947)
london =  (-0.4613, 51.4775)
haversine(vancouver, london)

## Projections

A map depicts geographic data on a flat, two-dimensional plane. The process of converting a three-dimensional surface to a two-dimensional plane is known as **projection**. There are many possible projections. Every projection distorts the surface in some way, so there is no "best" projection. The right projection depends on the situation.

One of the early map projections, developed during the Age of Exploration, was the **Mercator projection**, which is a type of _cylindrical projection_.

![](img/mercator.jpg)

To obtain a cylindrical projection, we first imagine rolling a flat piece of paper into a cylinder that encloses the Earth. Next, we imagine a source of light emanating from the center from the Earth, projecting each location on the surface of the earth onto the cylinder. The map is just the unrolled cylinder.

The Mercator projection is _conformal_, meaning that it preserves angles. This makes it useful for navigation. The problem with the Mercator projection is that it grossly distorts the sizes of locations near the poles. Canada, Russia, and Greenland appear much bigger than they actually are. Shown below is a comparison of the apparent sizes of Greenland and Africa in the Mercator projection and in actuality.

![](img/greenland-vs-africa.jpg)

There are other projections, the most notable ones being the Gall-Peters projection (a projection that preserves areas exactly) and the Robinson projection (a projection that is neither conformal nor equal-area, but a reasonable compromise between the two).

![](img/projections.jpg)

There are also _conic projections_. That is, locations on the surface of the earth are projected onto a cone instead of a cylinder.

![](img/conic.jpg)

The cone can touch the surface of the Earth at one latitude (in which case it is called a "tangent cone") or two (in which case it is called a "secant cone"). The latitudes at which the cone meets the surface of the Earth are called **reference latitudes**. Distances along the reference latitudes are preserved perfectly. A common conic projection that is used for regional maps is the **Lambert conformal conic** (LCC) projection.

For more information about map projections, read [Understanding Map Projections](http://downloads2.esri.com/support/documentation/ao_/710Understanding_Map_Projections.pdf). But let's take the projections that we know and start making maps.

# Making Maps with _cartopy_

To make maps, we use the _cartopy_ library, which plays well with _matplotlib_.

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
%matplotlib inline

The LCC projection is often used to make a map of the United States. For this projection, we have to specify the central latitude and longitude, as well as the reference latitudes (a.k.a. `standard_parallels=`). A list of all of the projections available in _cartopy_ can be found [here](https://scitools.org.uk/cartopy/docs/v0.16/crs/projections.html).

Once we have chosen a projection, it's time to add features, like bodies of water and country, to our map. To request a particular feature, we use the `.natural_earth_shp()` method, which tells _cartopy_ what data it should download from [Natural Earth](https://www.naturalearthdata.com/downloads/) and how to plot it.

For example, 
```
ax.natural_earth_shp(name="lakes",
                     resolution="110m",
                     category="physical", 
                     facecolor="skyblue")
```
tells _cartopy_ to download the `lakes` data from the [1:110m resolution physical vectors](https://www.naturalearthdata.com/downloads/110m-physical-vectors/) and to color the lakes skyblue.

In [ ]:
ax = plt.axes(
    projection=ccrs.LambertConformal(
        central_latitude=39,
        central_longitude=-96,
        standard_parallels=(33, 45)
    )
)

ax.natural_earth_shp(name="lakes",
                     resolution="110m",
                     category="physical", 
                     facecolor="skyblue")
ax.natural_earth_shp(name="ocean",
                     resolution="110m",
                     category="physical", 
                     facecolor="skyblue")

# To see that the name should be admin_0_countries, note that the URL is
# https://www.naturalearthdata.com/download/110m/cultural/ne_110m_admin_0_countries.zip
ax.natural_earth_shp(name="admin_0_countries",
                     resolution="110m",
                     category="cultural", 
                     facecolor="None", edgecolor="black")

# This sets the x- and y-axes limits
ax.set_extent([-125, -66.5, 20, 50])

Natural Earth provides other data, including states, provinces, land, and coastlines. A complete list can be found [here](https://www.naturalearthdata.com/downloads/).

# Exercises

1\. The latitudes and longitudes of 7322 world cities are contained in `https://dlsun.github.io/pods/data/worldcities.csv`. Find the 15 closest cities to St. Louis, as measured by Euclidean distance and Haversine distance. Do you notice any differences between the two lists?

2\. Make a map of Europe with country borders. (You will need to fiddle with the boundaries of the plot until it looks right.) Try different projection methods to see how they differ.